# SQL


Для анализа дан небольшой фаил размеров в 240 Мб содержащий только информацию о новостройках. Содержит срезы на каждый день(кроме дней поломок - простоя сервиса) Т.е. есть по каждому обекту есть информация за каждый день. Подучатеся в год(365 дней) информация об обект дублируется 365 раз. Учтите что сложный селект может работать не быстро.



Для анализа использовать фаил с базой /home/synchronize_data/price_analytics/24.db.copied

In [2]:
!ls -la /home/synchronize_data/price_analytics/24.db.copied

-rw-r--r-- 1 synchronize_data synchronize_data 254582784 Jun  4 08:29 /home/synchronize_data/price_analytics/24.db.copied


In [3]:
#pip install matplotlib
#sqlite3 /home/synchronize_data/price_analytics/24.db.copied


# Пример использования: задача вывести самую дорогую квартиру из имеющихся.

Запустите следующие 6 строчек, они должны отработать корректно.

In [4]:
import matplotlib.pyplot as plt  
import sqlite3
import json

In [5]:
connection = sqlite3.connect('/home/synchronize_data/price_analytics/24.db.copied')
cursor = connection.cursor()

In [6]:
cursor.execute("""
SELECT 
    *
FROM copied
where 
    category = "Новостройки" 
    or category ="Реновация"
ORDER BY price desc
LIMIT 1
""")

In [7]:
flat = cursor.fetchall()

In [8]:
flat

[(44871689,
  'Саввинская 27 | 27 | 2 | 340.9 | Саввинская 27 | 55.730824,37.557959 | 1',
  847419076,
  847419076,
  '2024-06-02 06:13:37.130681',
  'Новостройки',
  'level.ru',
  'https://level.ru/old/sav27/apartment/6room/5-1/',
  340.9,
  2,
  6,
  6,
  2485829)]

Поля рекомендованные:

id, "title", "price", "price_sale", date(datetime_create) as date, 
"brand", "apartment_area", "apartment_floor", "apartment_floors_total", "apartment_room", "apartment_ppm"

id: уникальный идентификатор квартиры в базе данных.

title: адрес квартиры.

price: цена квартиры.

price_sale: цена квартиры со скидкой (если есть).

date(datetime_create): дата и время создания записи о квартире.

brand: бренд или название компании, предлагающей эту квартиру.

apartment_area: площадь квартиры.

apartment_floor: номер этажа, на котором расположена квартира.

apartment_floors_total: общее количество этажей в доме.

apartment_room: количество комнат в квартире.

apartment_ppm: - цена за 1 м2

# Задачи:
sqlite3 /home/synchronize_data/price_analytics/24.db.copied\

.tables

SELECT * FROM copied limit 10;

In [9]:
# ставлю релоад вместо лоад чтобы можно было перезапускать весь ноутбук. комментарий на отдельной строчке, по другому не надо
#pip install ipython-sql

In [10]:
%reload_ext sql

In [11]:
%config SqlMagic.displaycon = False


In [12]:
%config SqlMagic.feedback = False

In [13]:
%sql sqlite:////home/synchronize_data/price_analytics/24.db.copied
#%sql sqlite:///24.db.copied

In [14]:
#далее можно вот так писать sql запросы и даже передавать результаты их выполнения в переменные - смотрите по ссылке вверху этого ноутбука
%sql SELECT * from copied WHERE id<600

id,title,price,price_sale,datetime_create,category,brand,url,apartment_area,apartment_floor,apartment_floors_total,apartment_room,apartment_ppm
598,"Причальный Bосток 29 65.3 Причальный пр., д. 8 55.76018,37.517482",37527037,26268925,2022-12-05 13:38:36.641439,Новостройки,level.ru,url_example,None,None,None,None,None
599,"Саввинская 17 17 4 82.5 55.734600731695195,37.565715889000494",202359478,202359478,2022-12-05 13:38:36.652957,Новостройки,level.ru,url_example,None,None,None,None,None


#status-доступна есть https://api.gkosnova.tech/public/api/v1/building-objects/17/properties?detail=0

1. Выведите список новостроек в порядке убывания количества доступных к покупке квартир.

2. Найдите среднюю цену квадратного метра по всем новостройкам

3. Выведите список новостроек в порядке убывания общего количества квартир.

4. Найдите среднюю площадь квартир в новостройках с количеством комнат больше 3

5. Найдите среднюю цену квадратного метра по всем новостройкам на конкретный день.

6. Найдите среднюю цен за квадратный метр в новостройках с количеством этажей больше 15.

7. Найти все новостройки которые появились в 2023 году

8. Найти все новостройки которые имеют больше 50 квартир

9. Найдите максимальную и минимальную цену среди всех новостроек

10. Найти все новостройки в которых есть квартиры площадью больше 100 кв метров

11. Найти новостройки котрые были построекны застройщщиком Донстрой

12. Найти всех застройщиков которые построили более 5 новостроек в 2023 году

13. Вывести список жилых комплексов с наибольшим количеством проданных квартир за последние 3 месяца. Вывести количество проданных квартир и процент от общего количества квартир этого комплекса

1. Выведите список новостроек в порядке убывания количества доступных к покупке квартир.


In [15]:
%%sql
SELECT 
    title, count(*) as кол_во_кв_продажа
FROM copied
WHERE 
    category = 'Новостройки'
    AND (price IS NOT NULL OR price_sale IS NOT NULL and price > 0)
GROUP BY title
ORDER BY 
    кол_во_кв_продажа DESC   
LIMIT 4;


title,кол_во_кв_продажа
1 1 1 -1 13.25 0,11450
14 1 1 -2 13.25 0,8273
10 1 1 -1 13.25 0,5192
14 1 1 -1 13.25 0,4847


2. Найдите среднюю цену квадратного метра по всем новостройкам.

In [16]:
%%sql
SELECT 
    avg(apartment_ppm) as average_price_per_meter
FROM copied
where 
    category = "Новостройки" 


average_price_per_meter
361418.78797071613


3. Выведите список новостроек в порядке убывания общего количества квартир.


In [17]:
%%sql
select title, count(*) as total_apartments
from copied
where category = 'Новостройки'
group by title
order by total_apartments desc
limit 7;


title,total_apartments
1 1 1 -1 13.25 0,11450
14 1 1 -2 13.25 0,8273
10 1 1 -1 13.25 0,5192
14 1 1 -1 13.25 0,4847
10 1 1 2 13.25 0,4253
10 1 1 1 13.25 0,4165
12 1 1 -2 13.25 0,3555


4. Найдите среднюю площадь квартир в новостройках с количеством комнат больше 3.

In [18]:
%%sql
select 
    round(avg(apartment_area),2) as average_area_flate
from copied
where
    category = "Новостройки" and apartment_room > 3


average_area_flate
119.94


5. Найдите среднюю цену квадратного метра по всем новостройкам на конкретный день.

In [19]:
%%sql
select 
    date(datetime_create), avg(apartment_ppm) as average_price_date
from copied
where
    category = "Новостройки" and date(datetime_create) = '2024-02-05'


date(datetime_create),average_price_date
2024-02-05,600898.7923076923


6. Найдите среднюю цен за квадратный метр в новостройках с количеством этажей больше 15.

In [20]:
%%sql
select
    round(avg(apartment_ppm),2) as average_ppm
from copied
where category = "Новостройки" and apartment_floors_total > 15


average_ppm
424140.18


7. Найти все новостройки которые появились в 2023 году

In [80]:
%%sql
select
    title, date(datetime_create) as date
from copied
where category = "Новостройки" and date like '2023%'
limit 5;


title,date
"Павелецкая Сити 9 2 23.8 ул. Дубининская, д. 59-69 55.71962004653103,37.63727228350035",2023-01-01
"Селигерская 1 3 17.8 пр-д Ильменский, вл. 12 55.859441,37.545155",2023-01-01
"Стрешнево Север 2 45.1 Волоколамское ш., д. 81, корп. 2 55.82495356307498,37.411886931513436",2023-01-01
"Южнопортовая 6 8 55.3 Г. Москва, ул. Южнопортовая, вл.28-28а 55.705727,37.696445",2023-01-01
"Южнопортовая 1 39 78.3 Г. Москва, ул. Южнопортовая, вл.28-28а 55.705727,37.696445",2023-01-01


8. Найти все новостройки которые имеют больше 50 квартир

In [86]:
%%sql 
select title, count(*) as кол_во_кв
from copied
where category = 'Новостройки'
group by title
having кол_во_кв > 50
limit 7



title,кол_во_кв
1 1 1 -1 13.25 0,11450
1 1 1 10 29.16 S,76
1 1 1 11 40.01 S,92
1 1 1 13 112.04 E4,92
1 1 1 13 70.55 2,93
1 1 1 14 40.02 S,92
1 1 1 14 70.55 2,74


9. Найдите максимальную и минимальную цену среди всех новостроек 

In [87]:
%%sql
select max(price), min(price) 
from copied 
where category = "Новостройки" and price != 0


max(price),min(price)
847419076,551000


10. Найти все новостройки в которых есть квартиры площадью больше 100 кв метров

In [90]:
%%sql
select *
from copied
where category = 'Новостройки' and apartment_area > 100
limit 5

id,title,price,price_sale,datetime_create,category,brand,url,apartment_area,apartment_floor,apartment_floors_total,apartment_room,apartment_ppm
649,"Саввинская 27 27 2 137.5 Саввинская 27 55.730669,37.557942",209072374,209072374,2022-12-05 13:50:46.382455,Новостройки,level.ru,https://level.ru/sav27/apartment/3room/4-1/,137.5,None,None,None,None
651,"Саввинская 27 27 3 139.5 Саввинская 27 55.730669,37.557942",238985532,238985532,2022-12-05 13:50:46.400989,Новостройки,level.ru,https://level.ru/sav27/apartment/3room/4-4/,139.5,None,None,None,None
653,"Саввинская 17 17 4 110.5 55.734600731695195,37.565715889000494",269713058,269713058,2022-12-05 13:50:46.465808,Новостройки,level.ru,https://level.ru/sav17/flat/3room/2-9/,110.5,None,None,None,None
655,"Саввинская 27 27 6 189.9 Саввинская 27 55.730669,37.557942",564747927,564747927,2022-12-05 13:50:46.527201,Новостройки,level.ru,https://level.ru/sav27/apartment/3room/1-11/,189.9,None,None,None,None
662,"Мичуринский 6 15 112.3 Озерная ул., вл. 7 55.687726,37.471704",59713584,41799508,2022-12-05 13:53:27.017773,Новостройки,level.ru,https://level.ru/michur/flat/6room/6-2-51/,112.3,None,None,None,None


11. Найти новостройки котрые были построекны застройщщиком Донстрой (# можно было title like "%donstroy%")

In [91]:
%%sql
select id, title, brand, date(datetime_create) as date
from copied
where category = 'Новостройки' and brand = 'donstroy.moscow'
limit 7

id,title,brand,date
29629804,"Символ Независимость Корпус 19, 2 секц 4 27.1",donstroy.moscow,2024-02-09
29629805,"Символ Независимость Корпус 19, 2 секц 3 28.3",donstroy.moscow,2024-02-09
29629806,"Символ Независимость Корпус 19, 2 секц 4 27.1",donstroy.moscow,2024-02-09
29629807,"Символ Независимость Корпус 19, 2 секц 3 29.9",donstroy.moscow,2024-02-09
29629808,"Символ Независимость Корпус 19, 2 секц 5 27.1",donstroy.moscow,2024-02-09
29629809,"Символ Независимость Корпус 19, 2 секц 5 29.4",donstroy.moscow,2024-02-09
29629810,"Символ Независимость Корпус 19, 2 секц 5 27.1",donstroy.moscow,2024-02-09


12. Найти всех застройщиков которые построили более 5 новостроек в 2023 году

In [92]:
%%sql
SELECT brand, count(*)
FROM copied
WHERE category = 'Новостройки' 
    AND datetime_create LIKE '2023%' 
GROUP BY brand
HAVING count(*) > 5


brand,count(*)
Capital Group,3932
level.ru,346656
Галс-Девелопмент,872


In [93]:
#12. Найти всех застройщиков которые построили более 5 новостроек в 2023 году
%sql SELECT brand, count(*) FROM copied WHERE category = 'Новостройки' and datetime_create LIKE '2023%' GROUP BY brand HAVING count(*) > 5

brand,count(*)
Capital Group,3932
level.ru,346656
Галс-Девелопмент,872


13. Вывести список жилых комплексов с наибольшим количеством проданных квартир за последние 3 месяца. Вывести количество проданных квартир и процент от общего количества квартир этого комплекса

In [32]:
# cursor.execute("""
# SELECT 
    
# FROM copied
# where 
#     category = "Новостройки" 
# """)
# cursor.fetchall()

In [95]:
#для себя посмотреть вывод табл кв за 2024 - столбцы
%sql select * from copied where datetime_create like '2024%' limit 4


id,title,price,price_sale,datetime_create,category,brand,url,apartment_area,apartment_floor,apartment_floors_total,apartment_room,apartment_ppm
26513533,"кв. 49_Вилиса Лациса ул., д. 38, к. 2 vilisa-latsisa-ul-d-38-k-1-2 8 57.4 Вилиса Лациса ул., д. 38, к. 1, 2",14200760,14200760,2024-01-20 05:33:23.754343,Реновация,fr.mos.ru,https://investmoscow.ru/tenders/tender/19151571,57.4,8,17,2,247400
26513534,"кв. 88_Вилиса Лациса ул., д. 38, к. 2 vilisa-latsisa-ul-d-38-k-1-2 12 57.2 Вилиса Лациса ул., д. 38, к. 1, 2",14294280,14294280,2024-01-20 05:33:23.763560,Реновация,fr.mos.ru,https://investmoscow.ru/tenders/tender/19151613,57.2,12,17,2,249900
26513535,"кв. 96_Вилиса Лациса ул., д. 38, к. 2 vilisa-latsisa-ul-d-38-k-1-2 13 57.4 Вилиса Лациса ул., д. 38, к. 1, 2",14344260,14344260,2024-01-20 05:33:23.771348,Реновация,fr.mos.ru,https://investmoscow.ru/tenders/tender/19146214,57.4,13,17,2,249900
26513536,"кв. 105_Вилиса Лациса ул., д. 38, к. 2 vilisa-latsisa-ul-d-38-k-1-2 15 57.6 Вилиса Лациса ул., д. 38, к. 1, 2",14394240,14394240,2024-01-20 05:33:23.779571,Реновация,fr.mos.ru,https://investmoscow.ru/tenders/tender/19146237,57.6,15,17,2,249900


# Задачи с обогащением:


1. Построить месячный график количества продаж новостроек за последний год.

2. Выведите список новостроек в западном районе.

3. Найти все новостройки в ЗАО и СЗАО которые имеют больше 10 этажей

4. Найти самый популярный размер квартир в каждом округе Москвы, и посчитать количеств продаж каждого размера ( сейчас за размер принимаем количество комнат)

5. Найти все новостройки в пределах заданного радиуса от опеределнного места. И определить расстояние от каждой новостройки до заданного места, распределив их по возрастанию расстояния до заданного места.

6. Построить график средней стоимости квадратного метра по месяцам, начиная с q2 2023 по q2 2024

7. Найти все новостройки в которых есть квартиры с видом на определенный объект ( например парк, реку, останкинскую телебашню, москва сити или другую достопримечательность).

8. Показать информацию о количестве квартир с таким видом их средние площади и стоимости.

9. Вывести список жилых комплексов с наибольшей разницей между стоимостью квартир в них и среденей стоимостью квартир в районе. Рассчитать процентное отклонение цены каждой квартиры от средней цены по району.

10. Начиная с 2023 года, построить несколько графиков по месяцам : стоимости кв метра квартир, курса доллара(сколько рублей за один доллар), ключевой ставки, уровень безработиц, численность населения в городе, среднее количество детей в семье, количество сделок по ипотеке, количество сделок по льготным ипотечным программам,

11. Найти все новостройки информация о которых появилась в 2023 году имеют рейтинг выше среднего по городу. Вывести список этих новостроек, среднее, минимальное и максимальное количество этажей в них, в разрезе типа квартир ( тип это количество комнат ) с количеством квартир.

12. Вывести весь жк квартиры в которых были введены в эксплуатацию с чистовой отделкой в 2024 году

13. вывести список жк бизнес класса в ВАО города Москвы района Измайлово, ключи от которых владельцы получили в 2023

14. Выведите жк где соотношение количества квартир к количеств парковочных мест меньше 2.5

15. Введите количество квартир в кирпичных домах города Москвы

16. Определите среднее время оживания лита в домах застройщика level

17. Выведите список жк от которых до ближайшей станции метро 10 минут пешком

18. Выведите общее число объектов застройщиков которые больше 20 лет на рынке, и при качестве объектов выше среднего.



In [ ]:
#1. Построить месячный график количества продаж новостроек за последний год.

# Задачи на подумать и предложить Выдвинуть 10 гипотез и проверить их на основе данных ( если данных не хватает то добавить данными из интернета)


In [ ]:
Гипотеза 1 - стоимость квартиры зависит от площади

Гипотеза 2 - стоимость квартиры зависит от расстояния до центра города

Гипотеза 3 -